# Loading the COVID-19 Dataset
Loads the [Open humdata COVID-19 dataset](https://data.humdata.org/dataset/novel-coronavirus-2019-ncov-cases) 
[Google APIs](https://storage.googleapis.com/covid19-open-data/v2/latest/main.csv).

In [2]:
import pandas as pd

# Loading URL with pandas prevents reading of records with key "NA" (Namibia) as NaN
data = pd.read_csv(
    "https://storage.googleapis.com/covid19-open-data/v2/latest/main.csv",
    keep_default_na=False,
    na_values=[""],
)

# NOTE: Latest downloads the last day of data, but you can download all by removing it.

# Print a part dataset
print(f"The dataset currently contains {len(data)} records, here is a sample:")
data.sample(5)

The dataset currently contains 22049 records, here is a sample:


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,key,date,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,locality_code,locality_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator,population,population_male,population_female,rural_population,urban_population,largest_city_population,clustered_population,population_density,...,hospital_beds,nurses,physicians,health_expenditure,out_of_pocket_health_expenditure,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,facial_coverings,vaccination_policy,stringency_index,noaa_station,noaa_distance,average_temperature,minimum_temperature,maximum_temperature,rainfall,snowfall,dew_point,relative_humidity
16364,PE_ANC_1301,2021-02-10,Q2697284,NaN,PE,Peru,ANC,Áncash,1301,Piscobamba,NaN,NaN,PE,PER,2,1.0,0.0,NaN,NaN,90.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3024.0,1436.0,1588.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.453110e+10,132.544936,22.634921,19.857143,25.269841,0.338667,NaN,19.031746,80.420139
9278,ES_CT_12087,2021-02-10,Q843223,NaN,ES,Spain,CT,Cataluña,12087,Palanques,NaN,NaN,ES,ESP,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.238100e+09,58.274787,11.798611,8.229167,16.750000,1.238250,99.06,7.388889,74.873040
19501,US_IN_18083,2021-02-10,Q512746,geoId/18083,US,United States of America,IN,Indiana,18083,Knox County,NaN,NaN,US,USA,2,2.0,1.0,NaN,105.0,3493.0,80.0,NaN,29767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37044.0,18689.0,18355.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-4.0,9.0,NaN,NaN,-9.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.253421e+10,18.117668,-4.661111,-8.966667,-1.500000,0.550333,30.48,-8.722222,74.716054
7062,CO_05_05380,2021-02-10,Q932102,NaN,CO,Colombia,05,Antioquia,5380,La Estrella,NaN,NaN,CO,COL,2,0.0,0.0,6.0,NaN,3846.0,54.0,3723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-11.0,-63.0,-6.0,NaN,-18.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.011010e+10,9.030433,22.322222,18.250000,28.250000,0.000000,NaN,18.155556,78.155355
18876,US_CA_06081,2021-02-10,Q108101,geoId/06081,US,United States of America,CA,California,06081,San Mateo County,NaN,NaN,US,USA,2,135.0,0.0,NaN,6304.0,36824.0,436.0,NaN,830327.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,768901.0,379940.0,388961.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-39.0,-21.0,-15.0,-76.0,-57.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.249389e+10,12.947285,11.984127,8.269841,16.730159,0.000000,NaN,6.357143,68.890601


To filter only country-level data from the dataset, look for records that have a `aggregation_level == 0` or, alternatively, null value for the `subregion1_code` (or `subregion1_name`) field:

In [3]:
# Look for rows with country level data
# Same as `data[data.subregion2_code.isna()]`
countries = data[data.aggregation_level == 0]

# Drop the subregion-level columns
countries = countries.drop(columns=['subregion1_code', 'subregion1_name', 'subregion2_code', 'subregion2_name'])

countries.tail()

,key,date,wikidata,datacommons,country_code,country_name,locality_code,locality_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator,population,population_male,population_female,rural_population,urban_population,largest_city_population,clustered_population,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,...,hospital_beds,nurses,physicians,health_expenditure,out_of_pocket_health_expenditure,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,facial_coverings,vaccination_policy,stringency_index,noaa_station,noaa_distance,average_temperature,minimum_temperature,maximum_temperature,rainfall,snowfall,dew_point,relative_humidity
21983,YE,2021-02-10,Q805,country/YEM,YE,Yemen,NaN,NaN,YE,YEM,0,0.0,0.0,NaN,NaN,2131.0,616.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29161922.0,14692284.0,14469638.0,18292399.0,10869523.0,2874373.0,2874373.0,53.977853,0.452,8172997.0,6846631.0,5799728.0,...,NaN,0.7852,NaN,72.039467,58.321163,33.0,49.0,48.0,21.0,2.0,-1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,27.78,4.113610e+10,238.027608,23.722222,12.111111,31.722222,0.000000,NaN,-0.055556,20.776269
21984,YT,2021-02-10,Q17063,country/MYT,YT,Mayotte,NaN,NaN,YT,MYT,0,0.0,0.0,NaN,NaN,10755.0,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256518.0,109092.0,115551.0,NaN,NaN,NaN,NaN,NaN,NaN,59419.0,53241.0,36178.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.700510e+10,16.041561,25.055556,23.944444,31.574074,0.084667,NaN,23.805556,81.805715
21985,ZA,2021-02-10,Q258,country/ZAF,ZA,South Africa,NaN,NaN,ZA,ZAF,0,2435.0,110.0,NaN,26055.0,1476135.0,46290.0,NaN,8495811.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58558270.0,28859303.0,29698967.0,19408553.0,39149717.0,5635127.0,20836302.0,47.630120,0.699,11887295.0,11028683.0,10950688.0,...,NaN,1.3078,0.9054,499.237549,38.775391,-19.0,0.0,-35.0,-43.0,-29.0,15.0,3.0,1.0,2.0,4.0,1.0,2.0,0.0,4.0,1.0,2.0,0.0,0.0,2.0,3.0,2.0,0.0,0.0,3.0,1.0,72.22,6.843810e+10,77.648653,23.645833,17.861111,32.729167,3.905250,NaN,15.951389,64.409527
22047,ZM,2021-02-10,Q953,country/ZMB,ZM,Zambia,NaN,NaN,ZM,ZMB,0,1236.0,14.0,NaN,10063.0,62663.0,853.0,NaN,973384.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17861030.0,8843212.0,9017818.0,9989317.0,7871713.0,2646619.0,2646619.0,23.341479,0.588,5799199.0,4507836.0,3303315.0,...,NaN,1.3376,1.1867,67.648666,8.015779,-1.0,10.0,1.0,-11.0,-22.0,8.0,2.0,2.0,1.0,4.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,0.0,0.0,3.0,0.0,49.07,6.766510e+10,155.975684,20.888889,18.000000,29.000000,0.000000,NaN,18.277778,85.029479
22048,ZW,2021-02-10,Q954,country/ZWE,ZW,Zimbabwe,NaN,NaN,ZW,ZWE,0,65.0,10.0,NaN,1064.0,34552.0,1326.0,NaN,336553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14645468.0,6983351.0,7662117.0,9928163.0,4717305.0,1521311.0,1521311.0,37.324591,0.535,4056369.0,3235339.0,2602286.0,...,NaN,1.9346,0.2096,110.149620,22.715431,-29.0,-9.0,-14.0,-44.0,-34.0,16.0,2.0,3.0,2.0,4.0,2.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,4.0,0.0,84.26,6.797510e+10,147.916284,14.833333,12.722222,19.277778,2.032000,NaN,12.222222,84.373404


To filter state/province data for a specific country, we need to look for records where the aggregation level is `1` (or where the region columns have non-null values). Below is an extraction of data related to Spain's subregions from the dataset:

In [5]:
# Filter records that have the right country code AND a non-null region code
# Same as `data[(data.country_code == 'ES') & ~(data.subregion`_code.isna())]`
spain_regions = data[(data.country_code == 'ES') & (data.aggregation_level == 1)]

# Drop municipality-level columns
spain_regions = spain_regions.drop(columns=['subregion2_code', 'subregion2_name'])

spain_regions.tail()

,key,date,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,locality_code,locality_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator,population,population_male,population_female,rural_population,urban_population,largest_city_population,clustered_population,population_density,human_development_index,population_age_00_09,...,hospital_beds,nurses,physicians,health_expenditure,out_of_pocket_health_expenditure,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,facial_coverings,vaccination_policy,stringency_index,noaa_station,noaa_distance,average_temperature,minimum_temperature,maximum_temperature,rainfall,snowfall,dew_point,relative_humidity
10178,ES_ML,2021-02-10,Q5831,nuts/ES64,ES,Spain,ML,Melilla,NaN,NaN,ES,ESP,1,0.0,NaN,NaN,NaN,5673.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84689.0,42793.0,41896.0,NaN,NaN,NaN,NaN,6049.9,NaN,14027.0,...,NaN,NaN,NaN,NaN,NaN,-26.0,-15.0,-31.0,-25.0,-18.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.033810e+10,0.820365,13.616667,10.411111,17.027778,0.685800,NaN,7.788889,68.145239
10179,ES_NC,2021-02-10,Q4018,nuts/ES22,ES,Spain,NC,Navarra,NaN,NaN,ES,ESP,1,0.0,0.0,NaN,NaN,49305.0,1041.0,NaN,NaN,0.0,4963.0,NaN,0.0,427.0,NaN,NaN,NaN,NaN,649946.0,321064.0,328882.0,NaN,NaN,NaN,NaN,62.5,0.918,65240.0,...,NaN,NaN,NaN,NaN,NaN,-28.0,5.0,2.0,-20.0,-12.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.085100e+09,5.201020,8.938272,6.049383,10.925926,1.301750,15.24,7.694444,91.511012
10180,ES_PV,2021-02-10,Q3995,nuts/ES21,ES,Spain,PV,País Vasco,NaN,NaN,ES,ESP,1,58.0,0.0,NaN,NaN,138629.0,3427.0,NaN,NaN,0.0,11008.0,NaN,0.0,895.0,NaN,NaN,NaN,NaN,2177880.0,1054101.0,1123779.0,NaN,NaN,NaN,NaN,302.4,0.924,197465.0,...,NaN,NaN,NaN,NaN,NaN,-49.0,3.0,4.0,-26.0,-20.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.080100e+09,14.175633,8.577778,5.411111,10.672222,2.743200,15.24,6.783951,88.937546
10181,ES_RI,2021-02-10,Q5727,nuts/ES23,ES,Spain,RI,La Rioja,NaN,NaN,ES,ESP,1,0.0,0.0,NaN,NaN,26507.0,665.0,NaN,NaN,2.0,2998.0,NaN,0.0,311.0,NaN,NaN,NaN,NaN,313571.0,154512.0,159059.0,NaN,NaN,NaN,NaN,62.3,NaN,29152.0,...,NaN,NaN,NaN,NaN,NaN,-61.0,-5.0,-9.0,-48.0,-21.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.084100e+09,26.140786,8.000000,5.361111,10.505556,3.668889,20.32,6.205556,88.959289
10182,ES_VC,2021-02-10,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,NaN,NaN,ES,ESP,1,0.0,0.0,NaN,NaN,333140.0,5152.0,NaN,NaN,0.0,26460.0,NaN,0.0,2265.0,NaN,NaN,NaN,NaN,4974969.0,2450794.0,2524175.0,NaN,NaN,NaN,NaN,214.4,NaN,467218.0,...,NaN,NaN,NaN,NaN,NaN,-50.0,-6.0,-10.0,-39.0,-26.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.284100e+09,23.036144,12.672840,7.641975,18.592593,0.592667,64.77,6.092593,65.163537


Below is a `key` column built using `country_code` for country-level data, `${country_code}_${subregion1_code}` for province/state level data, and `${country_code}_${subregion1_code}_${subregion2_code}` for municipality/county data:

In [ ]:
# Filter records for Spain at the country-level
spain_country = data[data.key == 'ES']

# Drop the subregion-level columns
spain_country = spain_country.drop(columns=['subregion1_code', 'subregion1_name', 'subregion2_code', 'subregion2_name'])

spain_country.tail()

In [ ]:
# Filter records for Madrid, one of the subregions of Spain
madrid = data[data.key == 'ES_MD']

madrid.tail()

In [4]:
# Load the epidemiology table
epi = pd.read_csv('https://storage.googleapis.com/covid19-open-data/v2/epidemiology.csv')

# Filter records for Madrid, one of the subregions of Spain
madrid = epi[epi.key == 'ES_MD']

madrid.tail()

,date,key,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested
4727632,2021-01-31,ES_MD,868.0,4.0,NaN,NaN,525537.0,12817.0,NaN,NaN
4742759,2021-02-01,ES_MD,4046.0,12.0,NaN,NaN,529583.0,12829.0,NaN,NaN
4757890,2021-02-02,ES_MD,4676.0,6.0,NaN,NaN,534259.0,12835.0,NaN,NaN
4773332,2021-02-03,ES_MD,3367.0,3.0,NaN,NaN,537626.0,12838.0,NaN,NaN
4788453,2021-02-04,ES_MD,0.0,0.0,NaN,NaN,537626.0,12838.0,NaN,NaN
